In [1]:
# LLM Models
from langchain_openai import ChatOpenAI

# Template
from langchain_core.prompts import ChatPromptTemplate

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio frontend
import gradio as gr

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
book_filepath = {"A Christmas Carol": '.\\data\\A_Christmas_Carol.txt',
                 "A Farewell to Arms": '.\\data\\A_Farewell_to_Arms.txt',
                 "Adventures of Huckleberry Finn": '.\\data\\Adventures_of_Huckleberry_Finn.txt',
                 "Crime and Punishment": '.\\data\\Crime_and_Punishment.txt',
                 "Moby Dick": '.\\data\\Moby_Dick.txt',
                 "Macbeth": '.\\data\\Macbeth.txt',
                 "Pride and Prejudice": '.\\data\\Pride_and_Prejudice.txt',
                 "Silas Marner": '.\\data\\Silas_Marner.txt',
                 "The Red Badge of Courage": '.\\data\\The_Red_Badge_of_Courage.txt',
                 "Wuthering Heights": '.\\data\\Wuthering_Heights.txt',
                 "Treasure Island": '.\\data\\Treasure_Island.txt'}

In [4]:
def book_chapter_summary(llm='gpt-4o-mini', book_title="Moby Dick", chapter_num=1):
    filepath = book_filepath[book_title]

    with open(file=filepath, mode='r') as file:
        f = file.read()

    f_lines = f.split('\n')
    # title = f_lines[0]
    # author = f_lines[2].split('By ')[-1]
    chapters = f.split('[divider]')[1:]

    num_chapters = len(chapters)

    if chapter_num > num_chapters:
        chapter_selected = num_chapters
    else:
        chapter_selected = chapter_num

    chapter = chapters[chapter_selected - 1]

    prompt = ChatPromptTemplate.from_template("""
    Provide a summary of the following text, at most 200 words. 
    Answer must be based only on the text given. 
    context: {chapter}
    """)

    if llm == 'gpt-4o-mini':
        chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                                max_completion_tokens=256,
                                temperature=0.5)
    elif llm == 'gpt-4.1-nano':
        chat_model = ChatOpenAI(model="gpt-4.1-nano",
                                max_completion_tokens=256,
                                temperature=0.5)

    chat_chain = prompt | chat_model | StrOutputParser()

    response = chat_chain.invoke({"chapter": chapters[chapter_num - 1]})

    return response

In [5]:
interface = gr.Interface(fn=book_chapter_summary,
                         inputs=[gr.Dropdown(choices=['gpt-4.1-nano', 'gpt-4o-mini'],
                                             value='gpt-4.1-nano',
                                             multiselect=False,
                                             label="LLM Model"),
                                 gr.Dropdown(choices=['A Christmas Carol',
                                                      'A Farewell to Arms',
                                                      'Adventures of Huckleberry Finn',
                                                      'Crime and Punishment',
                                                      'Moby Dick',
                                                      'Macbeth',
                                                      'Pride and Prejudice',
                                                      'Silas Marner',
                                                      'The Red Badge of Courage',
                                                      'Treasure Island',
                                                      'Wuthering Heights'],
                                             value='Moby Dick',
                                             multiselect=False,
                                             label="Title of the Book"),
                                 gr.Number(label="Chapter Number",
                                           value=1,
                                           minimum=1)],
                         outputs=[gr.Text(label="Summary")],
                         flagging_mode="never")

In [6]:
interface.launch(share=False)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [7]:
interface.close()

Closing server running on port: 7862
